In [271]:
from qutip import *
from math import sqrt, cos, sin
import numpy as np
from gen_state import make_W_list
from rho_methods import compute_witnesses, get_rho
import matplotlib.pyplot as plt
from tqdm import tqdm

In [252]:
H = basis(2,0) # ket 1
V = basis(2,1) # ket 2
D = 1/sqrt(2) * (H + V)
A = 1/sqrt(2) * (H - V)
R = 1/sqrt(2)*(H + 1j*V)
L = 1/sqrt(2)*(H - 1j*V)


In [260]:
def pauli_state(W_mat):
    """ Checks which pauli matrix combos make up the witness, W_mat

        0 - IxI 4 - XxI 8 - YxI 12 - ZxI
        1 - IxX 5 - XxX 9 - YxX 13 - ZxX
        2 - IxY 6 - XxY 10 - YxY 14 - ZxY
        3 - IxZ 7 - XxZ 11 - YxZ 15 - ZxZ
    """

    II = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]).flatten()
    IX = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,0,1,0]]).flatten()
    IY = np.array([[0,-1j,0,0],[1j,0,0,0],[0,0,0,-1j],[0,0,1j,0]]).flatten()
    IZ = np.array([[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]]).flatten()

    XI = np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]]).flatten()
    XX = np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]]).flatten()
    XY = np.array([[0,0,0,-1j],[0,0,1j,0],[0,-1j,0,0],[1j,0,0,0]]).flatten()
    XZ = np.array([[0,0,1,0],[0,0,0,-1],[1,0,0,0],[0,-1,0,0]]).flatten()

    YI = np.array([[0,0,-1j,0],[0,0,0,-1j],[1j,0,0,0],[0,1j,0,0]]).flatten()
    YX = np.array([[0,0,0,-1j],[0,0,1j,0],[0,1j,0,0],[1j,0,0,0]]).flatten()
    YY = np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]]).flatten()
    YZ = np.array([[0,0,-1j,0],[0,0,0,1j],[1j,0,0,0],[0,-1j,0,0]]).flatten()

    ZI = np.array([[1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,-1]]).flatten()
    ZX = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,-1],[0,0,-1,0]]).flatten()
    ZY = np.array([[0,-1j,0,0],[1j,0,0,0],[0,0,0,1j],[0,0,-1j,0]]).flatten()
    ZZ = np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]]).flatten()
    
    W_flat = W_mat.flatten() # for some reason this doesn't work as desired ... 4x4 matrix
    
    pauli_mats =  np.array([II,IX,IY,IZ,XI,XX,XY,XZ,YI,YX,YY,YZ,ZI,ZX,ZY,ZZ])
    contains_pauli = [False]*16

    for i in range(len(pauli_mats)):
        prod = np.dot(W_flat,pauli_mats[i])
        if prod != 0:
            contains_pauli[i] = True
    return contains_pauli

In [261]:
def scan_wit(scan_params):
    group_count = 0
    groups = {}
    
    for params in scan_params:
        alpha, beta, theta, phi, chi, eta, lam, gamma = params
    
        # Calculate W_state most general
        W_state = tensor(np.cos(theta)*H + np.sin(theta)*np.exp(1j*phi)*V, np.cos(alpha)*H + np.sin(alpha)*np.exp(1j*beta)*V) + tensor(-np.exp(-1j*chi)*np.sin(eta)*H + np.cos(eta)*V, -np.exp(-1j*lam)*np.sin(gamma)*H + np.cos(gamma)*V)

        # Calculate W
        W = partial_transpose(W_state*W_state.dag(), [0,1])

        # Extract Pauli matrices
        mat = W.full()
        
        pauli_mats = pauli_state(mat)
    
        # Store Pauli matrices in groups
        groups[group_count] = pauli_mats
        group_count += 1
    return groups

In [262]:
num_angles = 6

# Create 1D arrays for each parameter
alpha_L = np.linspace(0, np.pi, num_angles)
beta_L = np.linspace(0, 2*np.pi, num_angles)
theta_L = np.linspace(0, np.pi, num_angles)
phi_L = np.linspace(0, 2*np.pi, num_angles)
chi_L = np.linspace(0, 2*np.pi, num_angles)
eta_L = np.linspace(0, np.pi, num_angles)
lambda_L = np.linspace(0, 2*np.pi, num_angles)
gamma_L = np.linspace(0, np.pi, num_angles)

# Create mesh grids for each parameter
mesh_grids = np.meshgrid(alpha_L, beta_L, theta_L, phi_L, chi_L, eta_L, lambda_L, gamma_L, indexing='ij')

# Stack the mesh grids along the last axis to get a 6x6x6x6 mesh grid
scan_list = np.stack(mesh_grids, axis=-1).reshape(-1, 8)


In [263]:
groups = scan_wit(scan_list)

In [264]:
# now we check which states have the least measurments 
# if the measurments are already covered by a W or Wp group alone, get rid of them. Otherwise,
# add to a new dictionary
sorted_groups = {}

for wit in range(len(scan_list)):
    wp_meas = [groups[wit][11], groups[wit][14], groups[wit][7],groups[wit][13],groups[wit][6],groups[wit][9]]
    # change this to match what's in notebook

    
    # here it can get tricky, because we filter based on W'
    # option 1: make it so we elim if it is doing two W' measures and something else
    # option 2: check if it is doing at least one and add all to the count anyways
    if (groups[wit][11] and groups[wit][14]) or (groups[wit][7] and groups[wit][13]) \
        or (groups[wit][6] and groups[wit][9]):
        num_measurements = sum([1 for meas in wp_meas if meas]) - 2
            
        sorted_groups[wit] = num_measurements

sorted_groups = {k: v for k, v in sorted(sorted_groups.items(), key=lambda item: item[1])}


In [265]:
# keep only the ones that require 1 or two extra measurement
min_sorted_groups = {k: v for k, v in sorted_groups.items() if (v <= 2 and v > 0)}
print(len(min_sorted_groups))

151358


In [266]:
# Initialize ortho_min_groups
ortho_min_groups = {wit: min_sorted_groups[wit] for wit in min_sorted_groups}

# Initialize W_arrays for all witness indices
W_arrays = {}

# Calculate and store W_arrays for each witness index
for wit in min_sorted_groups:
    alpha, beta, theta, phi, chi, eta, lam, gamma = scan_list[wit]
    # Calculate W_state most general
    W_state = tensor(np.cos(theta)*H + np.sin(theta)*np.exp(1j*phi)*V, np.cos(alpha)*H + np.sin(alpha)*np.exp(1j*beta)*V) + tensor(-np.exp(-1j*chi)*np.sin(eta)*H + np.cos(eta)*V, -np.exp(-1j*lam)*np.sin(gamma)*H + np.cos(gamma)*V)
    # Calculate W
    W = partial_transpose(W_state*W_state.dag(), [0,1])
    mat = W.full()
    W_arrays[wit] = np.array(mat).flatten()

# Iterate through witness indices
for wit in min_sorted_groups:
    W_array = W_arrays[wit]

    # Check orthogonality with other witness indices
    for curr_wit in min_sorted_groups:
        if wit == curr_wit:
            continue
        
        curr_W_array = W_arrays[curr_wit]
        
        # Check orthogonality using dot product
        if np.dot(curr_W_array, W_array) == 0:
            del ortho_min_groups[wit]
            break

In [267]:
print(len(ortho_min_groups))

11157


In [268]:
# divide ortho_min_groups into subsets
num_sets = 5
set_size = len(ortho_min_groups)/ num_sets

ortho_min_groups_1 = {}
ortho_min_groups_2 = {}
ortho_min_groups_3 = {}
ortho_min_groups_4 = {}
ortho_min_groups_5 = {}

for index, (key, value) in enumerate(ortho_min_groups.items()):
    if index < set_size:
        ortho_min_groups_1[key] = value
    elif index > set_size and index <= 2*set_size:
        ortho_min_groups_2[key] = value
    elif index > 2*set_size and index <= 3*set_size:
        ortho_min_groups_3[key] = value
    elif index > 3*set_size and index <= 4*set_size:
        ortho_min_groups_4[key] = value
    else:
        ortho_min_groups_5[key] = value
        

In [ ]:
# Initialize num_witnessed dictionary
num_witnessed = {k: 0 for k, v in ortho_min_groups.items()}

# Define the number of intervals
num_intervals = 5

# Generate mesh grids for each parameter
alpha_L, beta_L, theta_L, phi_L, chi_L = np.meshgrid(np.linspace(0, np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      np.linspace(0, np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      indexing='ij')

# Flatten mesh grids
alpha_vals = alpha_L.flatten()
beta_vals = beta_L.flatten()
theta_vals = theta_L.flatten()
phi_vals = phi_L.flatten()
chi_vals = chi_L.flatten()



# Iterate over all combinations of parameters
for alpha, beta, theta, phi, chi in tqdm(zip(alpha_vals, beta_vals, theta_vals, phi_vals, chi_vals)):
    # Calculate entangled state
    ent_state = tensor(np.cos(theta)*H + np.sin(theta)*np.exp(1j*phi)*V,
                       np.cos(alpha)*H + np.sin(alpha)*np.exp(1j*beta)*V) + \
                np.exp(1j*chi) * tensor(-np.exp(-1j*phi)*np.sin(theta)*H + np.cos(theta)*V,
                                         -np.exp(-1j*beta)*np.sin(alpha)*H + np.cos(alpha)*V)
    rho = ket2dm(ent_state)

    # Iterate over witness indices
    for wit in ortho_min_groups_1:
        params = scan_list[wit]

        # Calculate W_state
        W_state = tensor(np.cos(params[2])*H + np.sin(params[2])*np.exp(1j*params[3])*V,
                         np.cos(params[0])*H + np.sin(params[0])*np.exp(1j*params[1])*V) + \
                  np.exp(1j*params[4]) * tensor(-np.exp(-1j*params[3])*np.sin(params[2])*H + np.cos(params[2])*V,
                                                 -np.exp(-1j*params[1])*np.sin(params[0])*H + np.cos(params[0])*V)
        
        # Calculate W
        W = partial_transpose(W_state*W_state.dag(), [0, 1])

        # Calculate expectation value
        exp_val = (W*rho).tr()


        if np.real(exp_val) < 0:
            num_witnessed[wit] += 1
                    
                    
                
                
                

222it [11:51,  2.78s/it]

In [242]:
# change to this version

# Initialize num_witnessed dictionary
num_witnessed = {k: 0 for k in ortho_min_groups}

# Define the number of intervals
num_intervals = 5

# Generate mesh grids for each parameter
mesh_grids = np.meshgrid(np.linspace(0, np.pi, num_intervals),
                         np.linspace(0, 2*np.pi, num_intervals),
                         np.linspace(0, np.pi, num_intervals),
                         np.linspace(0, 2*np.pi, num_intervals),
                         np.linspace(0, 2*np.pi, num_intervals),
                         indexing='ij')

# Flatten mesh grids
alpha_vals, beta_vals, theta_vals, phi_vals, chi_vals = [grid.flatten() for grid in mesh_grids]

# Calculate entangled state for each combination of parameters
ent_state = tensor(np.cos(theta_vals)*H + np.sin(theta_vals)*np.exp(1j*phi_vals)*V,
                   np.cos(alpha_vals)*H + np.sin(alpha_vals)*np.exp(1j*beta_vals)*V) + \
            np.exp(1j*chi_vals) * tensor(-np.exp(-1j*phi_vals)*np.sin(theta_vals)*H + np.cos(theta_vals)*V,
                                          -np.exp(-1j*beta_vals)*np.sin(alpha_vals)*H + np.cos(alpha_vals)*V)
rho = ket2dm(ent_state)

# Iterate over witness indices
for wit in ortho_min_groups:
    params = scan_list[wit]

    # Calculate W_state for each witness
    W_state = tensor(np.cos(params[2])*H + np.sin(params[2])*np.exp(1j*params[3])*V,
                     np.cos(params[0])*H + np.sin(params[0])*np.exp(1j*params[1])*V) + \
              np.exp(1j*params[4]) * tensor(-np.exp(-1j*params[3])*np.sin(params[2])*H + np.cos(params[2])*V,
                                             -np.exp(-1j*params[1])*np.sin(params[0])*H + np.cos(params[0])*V)

    # Calculate W
    W = partial_transpose(W_state*W_state.dag(), [0, 1])

    # Calculate expectation value
    exp_vals = (W*rho).tr()

    # Update num_witnessed if expectation value is less than 0
    num_witnessed[wit] += np.sum(np.real(exp_vals) < 0)

TypeError: One of inputs is not a quantum object